In [1]:
%config IPCompleter.greedy=True
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from nltk.translate.bleu_score import sentence_bleu
import scipy.sparse
import json

In [2]:
k_best = 5

def get_line_from_file_by_id(filename, id):
    with open(filename) as inf:
        for i, line in enumerate(inf):
            if i == id:
                return line
            
def get_line_from_filelist_by_id(filelist, id):
    return filelist[id]

with open('data/train.diff', 'r') as inf:
    train_diff_list = []
    for line in inf:
        train_diff_list.append(line)

with open('data/train.msg', 'r') as inf:
    train_msg_list = []
    for line in inf:
        train_msg_list.append(line)
            
#def extract_train_diff(id):
#    return get_line_from_file_by_id('data/train.diff', id)

def extract_train_diff(id):
    return get_line_from_filelist_by_id(train_diff_list, id)
            
#def extract_target_commit_message(id):
#    return get_line_from_file_by_id('data/train.msg', id)

def extract_target_commit_message(id):
    return get_line_from_filelist_by_id(train_msg_list, id)

In [22]:
new_diffs = []
with open('data/test.diff', 'r') as inf:
    for line in inf:
        new_diffs.append(line.strip())

In [23]:
%%time
#load vocabulary
with open('data/vocabulary.json', 'r') as inf:
    vocabulary = json.load(inf) 
vectorizer = CountVectorizer(vocabulary=vocabulary, token_pattern=r'\S+', stop_words=['<nl>'])

analyzer = vectorizer.build_analyzer()

#load bag-of-words matrix
train_bow_matrix = scipy.sparse.load_npz('data/bow_matrix.npz')
#train_bow_matrix.toarray()

CPU times: user 42.3 ms, sys: 28.8 ms, total: 71.1 ms
Wall time: 72.8 ms


In [24]:
%%time
newdiff_bow_matrix = vectorizer.transform(new_diffs)
#newdiff_bow_matrix.shape

CPU times: user 591 ms, sys: 6.66 ms, total: 597 ms
Wall time: 598 ms


In [25]:
%%time
cosine_matrix = cosine_distances(newdiff_bow_matrix, train_bow_matrix)
#cosine_matrix.shape

CPU times: user 41.2 s, sys: 1min 50s, total: 2min 31s
Wall time: 6min 15s


In [26]:
%%time
candidates_indices = [(row.argsort()[:k_best]).tolist() for row in cosine_matrix]
#len(candidates_indices[0]), len(candidates_indices)

CPU times: user 1min 4s, sys: 18.9 s, total: 1min 23s
Wall time: 1min 52s


In [27]:
cosine_matrix.shape

(12601, 64320)

In [28]:
best_bleus = []
best_diffs = []

def choose_best(diff, ids):
    best_bleu = 0.0
    best_id = ids[0]
    for id in ids:
        reference = extract_train_diff(id)
        score = sentence_bleu([analyzer(reference)], analyzer(diff))
        if score > best_bleu:
            best_bleu = score
            best_id = id
    best_bleus.append(best_bleu)
    best_diffs.append(extract_train_diff(best_id))
    return best_id

In [29]:
%%time
messages_id = []
for i, diff in enumerate(new_diffs):
    cur_id = choose_best(diff, candidates_indices[i])
    messages_id.append(cur_id)
    if i % 100 == 0:
        print(i)
    
#print(new_diffs, extract_train_diff(0), extract_train_diff(1), extract_train_diff(2), extract_train_diff(3))
    
#len(messages_id)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
CPU times: user 39.7 s, sys: 837 ms, total: 40.6 s
Wall time: 43.3 s


In [30]:
with open('data/generated.msg', 'w') as ouf:
    for i, message_id in enumerate(messages_id):
        message = extract_target_commit_message(message_id)
        ouf.write(message)
        #if i % 100 == 0:
        #    print(i)

In [31]:
best_bleus

[6.501553757486207e-155,
 1.0,
 0.7361167853656195,
 0.29166081361916846,
 1.0,
 0.663736437271196,
 0.8588553141228454,
 0.6540585844910979,
 0.7778111223054219,
 0.7969128121605122,
 0.041440964060130536,
 0.6540585844910979,
 0.8131501402127587,
 0.3532580652146734,
 0.7778111223054219,
 1.0,
 0.8020219183488043,
 0.7778111223054219,
 0.7751182834557346,
 0.693395566222006,
 0.49908106755981485,
 0.7778111223054219,
 0.4260070472007085,
 1.0,
 0.803154665668484,
 0.6540585844910979,
 0.4868932280584904,
 0.6580771759973243,
 0.6997522298221912,
 0.7447819789879647,
 0.34705559462738883,
 0.5383706724646368,
 0.6026080978557137,
 0.6699281464425185,
 1.0,
 0.7285062030354628,
 0.8403034716144353,
 0.5299928052715586,
 0.7912619863720214,
 0.8499508493439808,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.7641254083103469,
 0.7641254083103469,
 0.7641254083103469,
 0.10807159622222672,
 5.6573171863932095e-155,
 1.0,
 0.6026080978557137,
 0.7447819789879647,
 0.8390194026235064,
 0.4798425717074967

In [13]:
with open('data/best_diffs.txt', 'w') as ouf:
    for diff in best_diffs:
        ouf.write(diff + '\n')